# Import Libraries

In [71]:
import os
import json
import psycopg2
import pandas as pd
from sqlalchemy import create_engine, text, inspect

# Connect to PostgreSQL Database

In [72]:
with open("../config/credentials.json") as f:
    creds = json.load(f)

username = creds["POSTGRES_USERNAME"]
password = creds["POSTGRES_PASSWORD"]
host = creds["POSTGRES_HOST"]
port = creds["POSTGRES_PORT"]
dbname = creds["POSTGRES_DBNAME"]

DB_URL = f"postgresql://{username}:{password}@{host}:{port}/{dbname}"
engine = create_engine(DB_URL)

# View Database Schemas and Tables

In [13]:
# List all schemas and tables
with engine.connect() as conn:
    # List all schemas
    schemas = conn.execute(text("""
        SELECT schema_name
        FROM information_schema.schemata
        ORDER BY schema_name;
    """)).fetchall()

    # List all tables across all schemas
    tables = conn.execute(text("""
        SELECT table_schema, table_name
        FROM information_schema.tables
        WHERE table_type = 'BASE TABLE'
        ORDER BY table_schema, table_name;
    """)).fetchall()

# Display
print("Schemas:")
for schema in schemas:
    print(f" - {schema[0]}")

print("\nTables:")
for schema, table in tables:
    print(f" - {schema}.{table}")

Schemas:
 - information_schema
 - pg_catalog
 - public

Tables:
 - information_schema.sql_features
 - information_schema.sql_implementation_info
 - information_schema.sql_parts
 - information_schema.sql_sizing
 - pg_catalog.pg_aggregate
 - pg_catalog.pg_am
 - pg_catalog.pg_amop
 - pg_catalog.pg_amproc
 - pg_catalog.pg_attrdef
 - pg_catalog.pg_attribute
 - pg_catalog.pg_auth_members
 - pg_catalog.pg_cast
 - pg_catalog.pg_class
 - pg_catalog.pg_collation
 - pg_catalog.pg_constraint
 - pg_catalog.pg_conversion
 - pg_catalog.pg_database
 - pg_catalog.pg_db_role_setting
 - pg_catalog.pg_default_acl
 - pg_catalog.pg_depend
 - pg_catalog.pg_description
 - pg_catalog.pg_enum
 - pg_catalog.pg_event_trigger
 - pg_catalog.pg_extension
 - pg_catalog.pg_foreign_data_wrapper
 - pg_catalog.pg_foreign_server
 - pg_catalog.pg_foreign_table
 - pg_catalog.pg_index
 - pg_catalog.pg_inherits
 - pg_catalog.pg_init_privs
 - pg_catalog.pg_language
 - pg_catalog.pg_largeobject_metadata
 - pg_catalog.pg_namespa

# Create New Schema and Tables

In [17]:
with open("../config/credentials.json") as f:
    creds = json.load(f)

schema = "quant"

conn_str = f"host={host} port={port} dbname={dbname} user={username} password={password}"

# Load the schema.sql file
with open("../db/schema.sql", "r") as f:
    sql_script = f.read()

# Inject schema prefix
sql_script = sql_script.replace("CREATE TABLE ", f"CREATE TABLE {schema}.")

# === Connect and execute ===
with psycopg2.connect(conn_str) as conn:
    with conn.cursor() as cur:
        # Create schema first
        cur.execute(f"CREATE SCHEMA IF NOT EXISTS {schema};")
        print(f"Created or verified schema: {schema}")

        # Execute all table creation statements
        cur.execute(sql_script)
        print("Executed schema.sql and created all tables.")

Created or verified schema: quant
Executed schema.sql and created all tables.


# View Database Schemas and Tables

In [18]:
# List all schemas and tables
with engine.connect() as conn:
    # List all schemas
    schemas = conn.execute(text("""
        SELECT schema_name
        FROM information_schema.schemata
        ORDER BY schema_name;
    """)).fetchall()

    # List all tables across all schemas
    tables = conn.execute(text("""
        SELECT table_schema, table_name
        FROM information_schema.tables
        WHERE table_type = 'BASE TABLE'
        ORDER BY table_schema, table_name;
    """)).fetchall()

# Display
print("Schemas:")
for schema in schemas:
    print(f" - {schema[0]}")

print("\nTables:")
for schema, table in tables:
    print(f" - {schema}.{table}")

Schemas:
 - information_schema
 - pg_catalog
 - public
 - quant

Tables:
 - information_schema.sql_features
 - information_schema.sql_implementation_info
 - information_schema.sql_parts
 - information_schema.sql_sizing
 - pg_catalog.pg_aggregate
 - pg_catalog.pg_am
 - pg_catalog.pg_amop
 - pg_catalog.pg_amproc
 - pg_catalog.pg_attrdef
 - pg_catalog.pg_attribute
 - pg_catalog.pg_auth_members
 - pg_catalog.pg_cast
 - pg_catalog.pg_class
 - pg_catalog.pg_collation
 - pg_catalog.pg_constraint
 - pg_catalog.pg_conversion
 - pg_catalog.pg_database
 - pg_catalog.pg_db_role_setting
 - pg_catalog.pg_default_acl
 - pg_catalog.pg_depend
 - pg_catalog.pg_description
 - pg_catalog.pg_enum
 - pg_catalog.pg_event_trigger
 - pg_catalog.pg_extension
 - pg_catalog.pg_foreign_data_wrapper
 - pg_catalog.pg_foreign_server
 - pg_catalog.pg_foreign_table
 - pg_catalog.pg_index
 - pg_catalog.pg_inherits
 - pg_catalog.pg_init_privs
 - pg_catalog.pg_language
 - pg_catalog.pg_largeobject_metadata
 - pg_catalog.p

# Upload Ticker Metadata

In [77]:
engine = create_engine(f"postgresql://{username}:{password}@{host}:{port}/{dbname}")

# Load CSV
csv_path = "../data/final_tickers_date_sectors.csv"
df = pd.read_csv(csv_path)

print(f"Rows in CSV: {len(df)}")

# Upload to quant.ticker_metadata
df.to_sql("ticker_metadata", engine, schema=schema, if_exists="replace", index=False, method="multi")
print("Data uploaded to quant.ticker_metadata")

# Query the row count to verify
with engine.connect() as conn:
    result = conn.execute(text("SELECT COUNT(*) FROM quant.ticker_metadata"))
    count_in_db = result.scalar()

print(f"Rows in quant.ticker_metadata: {count_in_db}")

# Preview the first few rows
with engine.connect() as conn:
    preview = conn.execute(text("SELECT * FROM quant.ticker_metadata LIMIT 5")).fetchall()

print("Sample data from DB:")
for row in preview:
    print(row)

Rows in CSV: 647
Data uploaded to quant.ticker_metadata
Rows in quant.ticker_metadata: 647
Sample data from DB:
('A', '2017-01-01', '2025-06-22', 'XLV')
('AAL', '2017-01-01', '2024-09-22', 'XLI')
('AAP', '2017-01-01', '2023-08-24', 'XLY')
('AAPL', '2017-01-01', '2025-06-22', 'XLK')
('ABBV', '2017-01-01', '2025-06-22', 'XLV')


# Upload Macro Indicator Data

In [23]:
macro_folder = "../data/yfinance/macro"
table_name = "macro_indicators"
schema_name = "quant"

# Count and upload
all_rows = 0
all_files = [f for f in os.listdir(macro_folder) if f.endswith(".csv")]

for file in all_files:
    file_path = os.path.join(macro_folder, file)

    # Read CSV, skipping first 2 rows and manually naming columns
    df_raw = pd.read_csv(file_path, skiprows=2, names=["Date", "Close", "High", "Low", "Open", "Volume"])
    
    # Add Ticker column from filename
    ticker = file.replace(".csv", "")
    df_raw["Ticker"] = ticker

     # Convert to lowercase column names for PostgreSQL compatibility
    df_raw.columns = [col.lower() for col in df_raw.columns]
    df_raw = df_raw[["ticker", "date", "close", "high", "low", "open", "volume"]]

    # Convert date and clean
    df_raw["date"] = pd.to_datetime(df_raw["date"], format="%Y-%m-%d", errors='coerce').dt.date
    df_raw = df_raw.dropna(subset=["date", "close"])
    
    all_rows += len(df_raw)

    # Upload to PostgreSQL
    df_raw.to_sql(table_name, con=engine, schema=schema_name, if_exists="append", index=False, method="multi")

print(f"Uploaded {all_rows} total rows from {len(all_files)} files to {schema_name}.{table_name}")

# Final check
with engine.connect() as conn:
    count = conn.execute(text(f"SELECT COUNT(*) FROM {schema_name}.{table_name}")).scalar()
    print(f"Total rows in {schema_name}.{table_name}: {count}")
    
    sample = conn.execute(text(f"""
        SELECT * FROM {schema_name}.{table_name}
        ORDER BY Date ASC
        LIMIT 5;
    """)).fetchall()

print("Sample data:")
for row in sample:
    print(row)

Uploaded 16658 total rows from 7 files to quant.macro_indicators
Total rows in quant.macro_indicators: 16658
Sample data:
('5Y', datetime.date(2016, 1, 4), 1.7350000143051147, 1.7350000143051147, 1.6979999542236328, 1.7200000286102295, 0)
('DXY', datetime.date(2016, 1, 4), 98.87000274658205, 99.18000030517578, 98.0500030517578, 98.69000244140624, 0)
('10Y', datetime.date(2016, 1, 4), 2.244999885559082, 2.244999885559082, 2.200000047683716, 2.2300000190734863, 0)
('3M', datetime.date(2016, 1, 4), 0.1550000011920929, 0.1700000017881393, 0.152999997138977, 0.1650000065565109, 0)
('GOLD', datetime.date(2016, 1, 4), 1075.0999755859375, 1082.5, 1063.199951171875, 1063.4000244140625, 143)


# Upload Sector ETF OHLCV

In [25]:
etf_folder = "../data/tiingo/ohlcv"
table_name = "sector_etf_prices"
schema_name = "quant"

# Define the 11 sector ETFs
sector_etfs = {
    "XLB", "XLC", "XLE", "XLF", "XLI", "XLK",
    "XLP", "XLRE", "XLU", "XLV", "XLY"
}

# Upload Loop
all_rows = 0
all_files = [f for f in os.listdir(etf_folder) if f.endswith(".csv")]

for file in all_files:
    ticker = file.replace(".csv", "")
    if ticker not in sector_etfs:
        continue

    file_path = os.path.join(etf_folder, file)

    # Read CSV
    df_raw = pd.read_csv(file_path)

    # Parse and rename columns
    df_raw.columns = [col.lower() for col in df_raw.columns]
    df_raw["etf"] = ticker  # Add ETF name column

    # Convert date
    df_raw["date"] = pd.to_datetime(df_raw["date"], errors='coerce').dt.date
    df_raw = df_raw.dropna(subset=["date", "close"])

    # Reorder to match schema
    df_raw = df_raw[
        ["etf", "date", "close", "high", "low", "open", "volume",
         "adjclose", "adjhigh", "adjlow", "adjopen", "adjvolume",
         "divcash", "splitfactor"]
    ]

    all_rows += len(df_raw)

    # Upload
    df_raw.to_sql(table_name, con=engine, schema=schema_name, if_exists="append", index=False, method="multi")

print(f"Uploaded {all_rows} total rows from {len(sector_etfs)} sector ETF files to {schema_name}.{table_name}")

# Final DB Check
with engine.connect() as conn:
    count = conn.execute(text(f"SELECT COUNT(*) FROM {schema_name}.{table_name}")).scalar()
    print(f"Total rows in {schema_name}.{table_name}: {count}")

    sample = conn.execute(text(f"""
        SELECT * FROM {schema_name}.{table_name}
        ORDER BY date ASC
        LIMIT 5;
    """)).fetchall()

print("Sample data:")
for row in sample:
    print(row)

Uploaded 23408 total rows from 11 sector ETF files to quant.sector_etf_prices
Total rows in quant.sector_etf_prices: 23408
Sample data:
('XLE', datetime.date(2017, 1, 3), 76.17, 76.81, 75.36, 76.11, 24623100, 53.2964646707, 53.7442753231, 52.7297043139, 53.2544824221, 24623100, 0.0, 1.0)
('XLF', datetime.date(2017, 1, 3), 23.51, 23.67, 23.26, 23.61, 71259897, 20.082916608, 20.2195931992, 19.8693594344, 20.1683394775, 71259897, 0.0, 1.0)
('XLB', datetime.date(2017, 1, 3), 49.99, 50.27, 49.63, 49.83, 7737466, 42.3781205136, 42.6154854615, 42.0729370092, 42.2424834006, 7737466, 0.0, 1.0)
('XLC', datetime.date(2017, 1, 3), 76.959412, 77.458762, 75.927021, 76.44, 8408766, 28.468815, 28.770687, 28.104405, 28.394991, 14714247, 0.0, 1.0)
('XLI', datetime.date(2017, 1, 3), 62.59, 63.06, 62.35, 62.68, 21592200, 54.1442427058, 54.5508219369, 53.9366277793, 54.2220983033, 21592200, 0.0, 1.0)


# Upload Tiingo Ticker OHLCV

In [33]:
folder = "../data/tiingo/ohlcv"
table_name = "ohlcv_tiingo"
schema_name = "quant"

# Exclude these 11 sector ETFs
excluded_etfs = {
    "XLB", "XLC", "XLE", "XLF", "XLI", "XLK",
    "XLP", "XLRE", "XLU", "XLV", "XLY"
}

# Load additional ignore list
ignore_file = "../data/tiingo/tiingo_ignore_tickers.txt"
with open(ignore_file, "r") as f:
    ignored_tickers = set(line.strip() for line in f if line.strip())


# Combine both sets
excluded_tickers = excluded_etfs.union(ignored_tickers)

# Upload Loop
all_rows = 0
all_files = [f for f in os.listdir(etf_folder) if f.endswith(".csv")]
skipped_empty = []

for file in all_files:
    ticker = file.replace(".csv", "")
    if ticker in excluded_tickers:
        continue

    file_path = os.path.join(etf_folder, file)

    # Skip empty files
    if os.stat(file_path).st_size == 0:
        skipped_empty.append(ticker)
        continue

    try:
        df_raw = pd.read_csv(file_path)
    except pd.errors.EmptyDataError:
        skipped_empty.append(ticker)
        continue

    # Parse and rename
    df_raw.columns = [col.lower() for col in df_raw.columns]
    df_raw["ticker"] = ticker

    # Convert and clean
    df_raw["date"] = pd.to_datetime(df_raw["date"], errors='coerce').dt.date
    df_raw = df_raw.dropna(subset=["date", "close"])

    # Reorder to match schema
    df_raw = df_raw[
        ["ticker", "date", "close", "high", "low", "open", "volume",
         "adjclose", "adjhigh", "adjlow", "adjopen", "adjvolume",
         "divcash", "splitfactor"]
    ]

    all_rows += len(df_raw)

    # Upload
    df_raw.to_sql(table_name, con=engine, schema=schema_name, if_exists="append", index=False, method="multi")

print(f"Uploaded {all_rows} total rows from non-sector files to {schema_name}.{table_name}")
if skipped_empty:
    print(f"Skipped {len(skipped_empty)} empty files: {skipped_empty}")

# Final DB Check
with engine.connect() as conn:
    count = conn.execute(text(f"SELECT COUNT(*) FROM {schema_name}.{table_name}")).scalar()
    print(f"Total rows in {schema_name}.{table_name}: {count}")

    sample = conn.execute(text(f"""
        SELECT * FROM {schema_name}.{table_name}
        ORDER BY date ASC
        LIMIT 5;
    """)).fetchall()

print("Sample data:")
for row in sample:
    print(row)

Uploaded 1256197 total rows from non-sector files to quant.ohlcv_tiingo
Total rows in quant.ohlcv_tiingo: 1256197
Sample data:
('AAP', datetime.date(2017, 1, 3), 170.6, 171.36, 169.31, 170.78, 691526, 151.9407591699, 152.6176347676, 150.7918519053, 152.1010718114, 691526, 0.0, 1.0)
('AAPL', datetime.date(2017, 1, 3), 116.15, 116.33, 114.76, 115.8, 28781865, 26.8269201165, 26.8684943362, 26.505874753, 26.7460813559, 115127460, 0.0, 1.0)
('A', datetime.date(2017, 1, 3), 46.49, 46.75, 45.74, 45.93, 1739726, 43.6575100207, 43.9016690357, 42.9532051699, 43.1316290654, 1739726, 0.0, 1.0)
('AAL', datetime.date(2017, 1, 3), 46.3, 47.34, 46.135, 47.28, 6737752, 44.7427003579, 45.7477199771, 44.5832501298, 45.689738076, 6737752, 0.0, 1.0)
('ABBV', datetime.date(2017, 1, 3), 62.41, 63.03, 61.935, 62.92, 9328198, 43.4426100466, 43.8741822021, 43.1119700887, 43.7976129487, 9328198, 0.0, 1.0)


# Upload Yfinance Ticker OHLCV

In [34]:
folder = "../data/yfinance/ohlcv"
table_name = "ohlcv_yfinance"
schema_name = "quant"

# Upload Loop
all_rows = 0
skipped_empty = []
all_files = [f for f in os.listdir(folder) if f.endswith(".csv")]

for file in all_files:
    ticker = file.replace(".csv", "")
    file_path = os.path.join(folder, file)

    # Skip empty or unreadable files
    if os.stat(file_path).st_size == 0:
        skipped_empty.append(ticker)
        continue

    try:
        df_raw = pd.read_csv(file_path)
    except pd.errors.EmptyDataError:
        skipped_empty.append(ticker)
        continue

    # Rename + lowercase for DB compatibility
    df_raw.columns = [col.lower() for col in df_raw.columns]
    df_raw["ticker"] = ticker

    # Convert date and clean
    df_raw["date"] = pd.to_datetime(df_raw["date"], errors='coerce').dt.date
    df_raw = df_raw.dropna(subset=["date", "close"])

    # Reorder to match schema
    df_raw = df_raw[
        ["ticker", "date", "close", "high", "low", "open", "volume",
         "adjclose", "adjhigh", "adjlow", "adjopen", "adjvolume",
         "divcash", "splitfactor"]
    ]

    all_rows += len(df_raw)

    # Upload
    df_raw.to_sql(table_name, con=engine, schema=schema_name,
                  if_exists="append", index=False, method="multi")

print(f"Uploaded {all_rows} total rows from {len(all_files)} files to {schema_name}.{table_name}")
if skipped_empty:
    print(f"Skipped {len(skipped_empty)} empty files: {skipped_empty}")

# Final DB Check
with engine.connect() as conn:
    count = conn.execute(text(f"SELECT COUNT(*) FROM {schema_name}.{table_name}")).scalar()
    print(f"Total rows in {schema_name}.{table_name}: {count}")

    sample = conn.execute(text(f"""
        SELECT * FROM {schema_name}.{table_name}
        ORDER BY date ASC
        LIMIT 5;
    """)).fetchall()

print("Sample data:")
for row in sample:
    print(row)

Uploaded 8512 total rows from 4 files to quant.ohlcv_yfinance
Total rows in quant.ohlcv_yfinance: 8512
Sample data:
('TKO', datetime.date(2017, 1, 3), 18.239999771118164, 18.56999969482422, 18.0, 18.56999969482422, 606000, 16.363698959350586, 18.56999969482422, 18.0, 18.56999969482422, 606000, 0.0, 1.0)
('VTRS', datetime.date(2017, 1, 3), 39.11000061035156, 39.27999877929688, 38.41999816894531, 38.47999954223633, 5031900, 32.707218170166016, 39.27999877929688, 38.41999816894531, 38.47999954223633, 5031900, 0.0, 1.0)
('DOC', datetime.date(2017, 1, 3), 29.780000686645508, 29.920000076293945, 29.520000457763672, 29.8799991607666, 3083500, 19.266828536987305, 29.920000076293945, 29.520000457763672, 29.8799991607666, 3083500, 0.0, 1.0)
('LIN', datetime.date(2017, 1, 3), 116.9000015258789, 118.22000122070312, 116.12000274658205, 117.97000122070312, 2364200, 101.37499237060548, 118.22000122070312, 116.12000274658205, 117.97000122070312, 2364200, 0.0, 1.0)
('DOC', datetime.date(2017, 1, 4), 30

# Add Ticker Metadata for Yfinance OHLCV

In [78]:
# Define the 4 entries
new_entries = pd.DataFrame([
    {"Ticker": "DOC", "StartDate": "2017-01-01", "EndDate": "2025-06-22", "SectorETF": "XLRE"},
])

# Convert dates
new_entries["StartDate"] = pd.to_datetime(new_entries["StartDate"]).dt.date
new_entries["EndDate"] = pd.to_datetime(new_entries["EndDate"]).dt.date

# Get row count before insert
with engine.connect() as conn:
    initial_count = conn.execute(text("SELECT COUNT(*) FROM quant.ticker_metadata")).scalar()

# Insert the 4 rows
new_entries.to_sql(
    name="ticker_metadata",
    con=engine,
    schema="quant",
    if_exists="append",
    index=False,
    method="multi"
)

# Get row count after insert
with engine.connect() as conn:
    final_count = conn.execute(text("SELECT COUNT(*) FROM quant.ticker_metadata")).scalar()
    inserted_rows = conn.execute(text("""
        SELECT * FROM quant.ticker_metadata
        WHERE "Ticker" IN ('DOC', 'LIN', 'TKO', 'VTRS')
        ORDER BY "Ticker"
    """)).fetchall()

# Output
print(f"Initial row count: {initial_count}")
print(f"Final row count: {final_count}")
print(f"Row count increased by: {final_count - initial_count}")


Initial row count: 647
Final row count: 648
Row count increased by: 1


# Combine OHLCV Data Tables & Delete Old Tables

In [67]:
# Create the new combined table
with engine.begin() as conn:
    conn.execute(text("""
        CREATE TABLE IF NOT EXISTS quant.ticker_ohlcv (
            Ticker TEXT,
            Date DATE,
            Close FLOAT,
            High FLOAT,
            Low FLOAT,
            Open FLOAT,
            Volume BIGINT,
            AdjClose FLOAT,
            AdjHigh FLOAT,
            AdjLow FLOAT,
            AdjOpen FLOAT,
            AdjVolume BIGINT,
            DivCash FLOAT,
            SplitFactor FLOAT,
            PRIMARY KEY (Ticker, Date)
        );
    """))

# Get original row counts
with engine.connect() as conn:
    count_tiingo = conn.execute(text("SELECT COUNT(*) FROM quant.ohlcv_tiingo")).scalar()
    count_yfinance = conn.execute(text("SELECT COUNT(*) FROM quant.ohlcv_yfinance")).scalar()

print(f"Tiingo rows: {count_tiingo}")
print(f"YFinance rows: {count_yfinance}")

# Insert data into new table
with engine.begin() as conn:
    conn.execute(text("INSERT INTO quant.ticker_ohlcv SELECT * FROM quant.ohlcv_tiingo"))
    conn.execute(text("INSERT INTO quant.ticker_ohlcv SELECT * FROM quant.ohlcv_yfinance"))

# Validate row count
with engine.connect() as conn:
    total_new = conn.execute(text("SELECT COUNT(*) FROM quant.ticker_ohlcv")).scalar()

print(f"Combined rows: {total_new} (Expected: {count_tiingo + count_yfinance})")

# Drop old tables ONLY if row count is correct
if total_new == count_tiingo + count_yfinance:
    with engine.begin() as conn:
        conn.execute(text("DROP TABLE quant.ohlcv_tiingo"))
        conn.execute(text("DROP TABLE quant.ohlcv_yfinance"))
    print("Old tables dropped successfully.")
else:
    print("Row mismatch. Tables not dropped. Please check for duplicates or conflicts.")

Tiingo rows: 1256197
YFinance rows: 8512
Combined rows: 1264709 (Expected: 1264709)
Old tables dropped successfully.


# Overview of Database Schema

In [79]:
inspector = inspect(engine)
quant_tables = inspector.get_table_names(schema="quant")

print(f"Tables in 'quant' schema ({len(quant_tables)} total):\n")

with engine.connect() as conn:
    for table in quant_tables:
        # Get column info
        columns = inspector.get_columns(table, schema="quant")
        col_info = [(col["name"], str(col["type"])) for col in columns]
        num_cols = len(col_info)
        
        # Get row count
        row_count = conn.execute(text(f"SELECT COUNT(*) FROM quant.{table}")).scalar()
        
        print(f"Table: {table}")
        print(f"Rows: {row_count}")
        print(f"Columns: {num_cols}")
        for name, dtype in col_info:
            print(f"     - {name}: {dtype}")
        print("-" * 50)

Tables in 'quant' schema (4 total):

Table: ticker_ohlcv
Rows: 1264709
Columns: 14
     - ticker: TEXT
     - date: DATE
     - close: DOUBLE PRECISION
     - high: DOUBLE PRECISION
     - low: DOUBLE PRECISION
     - open: DOUBLE PRECISION
     - volume: BIGINT
     - adjclose: DOUBLE PRECISION
     - adjhigh: DOUBLE PRECISION
     - adjlow: DOUBLE PRECISION
     - adjopen: DOUBLE PRECISION
     - adjvolume: BIGINT
     - divcash: DOUBLE PRECISION
     - splitfactor: DOUBLE PRECISION
--------------------------------------------------
Table: sector_etf_prices
Rows: 23408
Columns: 14
     - etf: TEXT
     - date: DATE
     - close: DOUBLE PRECISION
     - high: DOUBLE PRECISION
     - low: DOUBLE PRECISION
     - open: DOUBLE PRECISION
     - volume: BIGINT
     - adjclose: DOUBLE PRECISION
     - adjhigh: DOUBLE PRECISION
     - adjlow: DOUBLE PRECISION
     - adjopen: DOUBLE PRECISION
     - adjvolume: BIGINT
     - divcash: DOUBLE PRECISION
     - splitfactor: DOUBLE PRECISION
------